In [6]:

from pyspark.sql import SparkSession
#from pyspark.sql.functions import *
spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('Ready_for_analysis').getOrCreate()

In [7]:

#IMPORTING SETUP PACKAGES
import statsmodels.formula.api as smf
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 500)



In [8]:
number_of_top_items = '5000'
plot_name = number_of_top_items[0] + 'k'

#print(plot_name)

df = pd.read_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/Top'+number_of_top_items+'_seller_YTD_group_by_item_visit_20210226.pickle')


In [9]:
print(df.item_code.nunique())


df_final_list= pd.read_csv('./pickle_files/df_final_spark_top5k_080421.csv',index_col=0)

4488


In [37]:
df_final_list.dropna(inplace=True)

In [39]:
df_final_list

,item_code,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey
0,1000012530,0.016220,99.99,2021-02-13,66.081387,-0.0187,0.0059,True,44.680,247523697279378
1,1000012794,0.014916,259.99,2021-02-25,41.525974,-0.0120,0.0052,False,141.050,151936474932722
3,1000022179,0.003121,479.99,2021-02-06,93.258739,-0.0136,0.0033,True,276.870,11954970507506
4,1000024320,0.024532,119.99,2020-12-01,22.354839,-0.0220,0.0097,False,65.109,31827848594342
5,1000034743,0.005481,279.99,2021-02-04,110.202494,-0.0186,0.0048,True,171.030,206871238538442
...,...,...,...,...,...,...,...,...,...,...
4176,1000176709,0.036867,99.99,2020-11-20,186.000000,-0.0225,0.0057,True,37.259,106298526077769
4177,1000176740,0.027844,99.99,2020-12-22,49.038462,-0.0136,0.0045,True,33.060,89073329847717
4178,1000179473,0.016499,104.99,2021-02-20,73.510638,-0.0374,0.0100,True,31.120,142441836666160
4179,1000182823,0.005275,529.99,2021-02-25,190.994048,-0.0140,0.0031,True,156.780,24335138052163


In [40]:
print(df.dtypes)

min_date                  object
max_date                  object
item_code                 object
item_price               float64
delivery_weeks             int64
avg_sales                float64
avg_unique_views         float64
conv_rate                float64
bin                        int32
item_parent_item_code     object
item_main_category        object
item_sub_category_1       object
item_sub_category_2       object
log_price                float64
log_delivery_weeks       float64
log_sales                float64
log_avg_unique_views     float64
log_conv_rate            float64
dtype: object


In [41]:
df_final_list.dtypes

item_code             int64
avg_conv_rate       float64
last_price          float64
last_date            object
avg_unique_views    float64
PE                  float64
PE_err              float64
has_enough_stat        bool
pc2                 float64
item_skey             int64
dtype: object

In [42]:
df_items = df_final_list['item_code'].tolist()

In [43]:
str_list_items = []
for i in df_items:
    i = '00000000'+str(i)
    str_list_items.append(i)


In [44]:
df_filt = df.loc[df['item_code'].isin(str_list_items)]

In [46]:
import numpy as np
df_filt.head()#['log_avg_unique_views']= np.log(df_filt['avg_unique_views'])

,min_date,max_date,item_code,item_price,delivery_weeks,avg_sales,avg_unique_views,conv_rate,bin,item_parent_item_code,item_main_category,item_sub_category_1,item_sub_category_2,log_price,log_delivery_weeks,log_sales,log_avg_unique_views,log_conv_rate
0,2020-01-01,2020-05-23,000000001000012530,99.99,1,0.520833,35.743056,0.014573,144,000000008000000774,ESSZIMMERSTÜHLE&BÄNKE,BÄNKE,BÄNKE,4.605071,0.000100,-0.652133,3.576356,-4.221842
1,2020-01-01,2020-01-02,000000001000012794,219.99,2,0.000000,27.000000,0.000001,2,000000008000000826,BETTEN,BETTGESTELLE,MASSIVHOLZBETTEN,5.393583,0.693197,-9.210340,3.295837,-9.210340
3,2020-01-01,2020-01-01,000000001000022179,549.99,3,0.000000,50.000000,0.000001,1,000000008000001920,BETTEN,BETTGESTELLE,POLSTERBETTEN,6.309900,1.098646,-9.210340,3.912023,-9.210340
4,2020-01-01,2020-01-01,000000001000024320,99.99,1,0.000000,8.000000,0.000001,1,000000008000002133,MATRATZEN&LATTENROSTE,MATRATZEN,KALTSCHAUMMATRATZEN,4.605071,0.000100,-9.210340,2.079442,-9.210340
5,2020-01-01,2020-01-03,000000001000034743,249.99,5,0.000000,42.333333,0.000001,3,000000008000003627,POLSTER,SESSEL,LOUNGESESSEL,5.521421,1.609458,-9.210340,3.745575,-9.210340


In [47]:
wls_cr_with_fe = smf.wls("log_avg_unique_views ~ log_price + log_delivery_weeks + C(item_code)", df_filt, weights=df_filt['bin']).fit()
wls_cr_with_fe.summary()
#beta_log_price, beta_std_err_log_price = wls_cr_with_fe.params[-2].round(4),wls_cr_with_fe.bse[-2].round(4)
#beta_log_del, beta_std_err_log_del = wls_cr_with_fe.params[-1].round(4),wls_cr_with_fe.bse[-1].round(4)
#Result_log_price = 'Log_price  = ' + str(beta_log_price) + ' ± ' + str(beta_std_err_log_price)
#Result_log_del = 'Log_del_week = ' + str(beta_log_del) + ' ± ' + str(beta_std_err_log_del)

<class 'statsmodels.iolib.summary.Summary'>
"""
                             WLS Regression Results                             
================================================================================
Dep. Variable:     log_avg_unique_views   R-squared:                       0.795
Model:                              WLS   Adj. R-squared:                  0.791
Method:                   Least Squares   F-statistic:                     192.8
Date:                  Mon, 12 Apr 2021   Prob (F-statistic):               0.00
Time:                          20:54:55   Log-Likelihood:            -2.6504e+05
No. Observations:                211704   AIC:                         5.384e+05
Df Residuals:                    207534   BIC:                         5.812e+05
Df Model:                          4169                                         
Covariance Type:              nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              3.0430      0.086     35.218      0.000       2.874       3.212
C(item_code)[T.000000001000002470]     0.2177      0.115      1.887      0.059      -0.008       0.444
C(item_code)[T.000000001000002471]     0.2075      0.115      1.798      0.072      -0.019       0.434
C(item_code)[T.000000001000002479]     0.5913      0.115      5.127      0.000       0.365       0.817
C(item_code)[T.000000001000002542]     0.4997      0.116      4.309      0.000       0.272       0.727
C(item_code)[T.000000001000003447]    -0.2026      0.116     -1.747      0.081      -0.430       0.025
C(item_code)[T.000000001000004855]     1.1481      0.116      9.932      0.000       0.922       1.375
C(item_code)[T.000000001000005080]    -0.6780      0.116     -5.862      0.000      -0.905      -0.451
C(item_code)[T.000000001000005089]    -0.4271      0.116     -3.693      0.000      -0.654      -0.200
C(item_code)[T.000000001000005090]     0.1230      0.116      1.063      0.288      -0.104       0.350
C(item_code)[T.000000001000005106]     0.4940      0.116      4.271      0.000       0.267       0.721
C(item_code)[T.000000001000005133]    -0.1133      0.116     -0.980      0.327      -0.340       0.113
C(item_code)[T.000000001000005134]     0.0459      0.116      0.397      0.692      -0.181       0.273
C(item_code)[T.000000001000005135]    -0.6540      0.116     -5.653      0.000      -0.881      -0.427
C(item_code)[T.000000001000005136]     0.2246      0.116      1.942      0.052      -0.002       0.451
C(item_code)[T.000000001000005176]     0.2164      0.116      1.873      0.061      -0.010       0.443
C(item_code)[T.000000001000005177]     1.8967      0.116     16.412      0.000       1.670       2.123
C(item_code)[T.000000001000005178]     1.4022      0.116     12.124      0.000       1.176       1.629
C(item_code)[T.000000001000005179]     1.0296      0.116      8.898      0.000       0.803       1.256
C(item_code)[T.000000001000005180]     0.6107      0.116      5.275      0.000       0.384       0.838
C(item_code)[T.000000001000005185]    -0.8154      0.116     -7.051      0.000      -1.042      -0.589
C(item_code)[T.000000001000005189]    -0.1848      0.116     -1.600      0.110      -0.411       0.042
C(item_code)[T.000000001000005192]     0.1358      0.116      1.175      0.240      -0.091       0.362
C(item_code)[T.000000001000005202]    -0.3184      0.116     -2.744      0.006      -0.546      -0.091
C(item_code)[T.000000001000005211]    -0.6191      0.116     -5.343      0.000      -0.846      -0.392
C(item_code)[T.000000001000005213]    -0.8243      0.116     -7.133      0.000      -1.051      -0.598
C(item_code)[T.000000001000005221]    -0.2017      0.115     -1.750      0